<a href="https://colab.research.google.com/github/anjelisa01/resume_scoring_app/blob/main/sectioned_deeper_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
from transformers import pipeline


#functions

##milih section penting

In [2]:
def select_relevant_sections(resume_dict, jd_dict):
    selected_resume = {k: resume_dict[k] for k in ['summary', 'skills', 'experience', 'projects'] if k in resume_dict}
    selected_jd = {k: jd_dict[k] for k in ['summary', 'requirements', 'responsibilities', 'preferred_qualifications'] if k in jd_dict}
    return selected_resume, selected_jd

##ringkas sections

In [3]:
# def summarize_sections(section_dict, summarizer, max_tokens=100):
#     summarized = {}
#     for section, content in section_dict.items():
#         summary = summarizer(content, max_length=max_tokens, min_length=30, do_sample=False)[0]['summary_text']
#         summarized[section] = summary
#     return summarized

In [4]:
def summarize_sections(section_dict, summarizer):
    summarized = {}

    for section, content in section_dict.items():
        if not content.strip():
            summarized[section] = ""
            continue

        if section.lower() == "summary":
            # Aim for ~2 sentences
            summary = summarizer(content, max_length=60, min_length=30, do_sample=False)[0]['summary_text']
            summarized[section] = summary

        elif section.lower() == "skills":
            # Extract & clean skill list (assume comma or newline separated)
            skills = [skill.strip() for skill in content.replace('\n', ',').split(',') if skill.strip()]
            summarized[section] = ", ".join(skills)

        elif section.lower() in ["experience", "project"]:
            # Use higher max length to keep important content
            summary = summarizer(content, max_length=150, min_length=60, do_sample=False)[0]['summary_text']
            summarized[section] = summary

        else:
            # Default summarization
            summary = summarizer(content, max_length=80, min_length=30, do_sample=False)[0]['summary_text']
            summarized[section] = summary

    return summarized


##Analyzed match per sections

In [5]:
# def analyze_match(resume_summaries, jd_summaries, llm_pipeline):
#     analyses = {}
#     section_map = {
#         'summary': 'summary',
#         'skills': 'requirements',
#         'experience': 'responsibilities',
#         'projects': 'preferred_qualifications'
#     }
#     for resume_sec, jd_sec in section_map.items():
#         if resume_sec in resume_summaries and jd_sec in jd_summaries:
#             prompt = (
#                 f"Candidate {resume_sec.capitalize()}:\n{resume_summaries[resume_sec]}\n\n"
#                 f"Job {jd_sec.capitalize()}:\n{jd_summaries[jd_sec]}\n\n"
#                 f"Q: How well does the candidate's {resume_sec} align with the job's {jd_sec}?\nA:"
#             )
#             result = llm_pipeline(prompt, max_new_tokens=150, do_sample=False)[0]['generated_text']
#             analyses[resume_sec] = result.replace(prompt, '').strip()
#     return analyses

##Final output

In [6]:
# def compile_analysis(contact_info, analysis_results):
#     output = f"Candidate: {contact_info}\n\n"
#     for section, analysis in analysis_results.items():
#         output += f"{section.capitalize()} Match Analysis:\n{analysis}\n\n"
#     return output.strip()



#Test usage

In [7]:
resume={'contact': 'Alex Johnson (123) 456-7890, alexjohnson. dev@email. com, github. com/alexjohnson-dev, alexjohnson. dev.',
 'summary': 'Junior Web Developer with a strong foundation in front-end and back-end technologies. Skilled in building responsive, user-centric websites and applications. Passionate about learning new technologies and delivering high-quality code.',
 'skills': 'This candidate is proficient in Languages: HTML5, CSS3, JavaScript (ES6+), Python Frameworks/Libraries: React. js, Node. js, Express. js, Bootstrap Databases: MongoDB, MySQL Tools: Git, GitHub, Visual Studio Code, Figma, Postman Other: RESTful APIs, Responsive Design, and Agile Methodology.',
 'experience': 'Web Development Intern, BrightWave Solutions, Remote, Jan 2024-Apr 2024-Developed and maintained front-end components using React. js and Bootstrap. Built RESTful APIs with Node. js and Express. js. Created mobile-first responsive designs with UX teams. Improved website performance by 20%. Freelance Web Developer, Self-Employed, Remote, May 2023-Dec 2023-Designed and developed custom websites for local businesses. Integrated third-party APIs (Google Maps, Stripe). Provided maintenance and support for client websites.',
 'education': 'Bachelor of Science in Computer Science, University of Springfield, Springfield, IL, May 2023 Relevant Coursework: Web Development, Database Systems, Software Engineering, Human-Computer Interaction.',
 'certifications': 'I hold certifications including Responsive Web Design Certification (freeCodeCamp, 2022)-JavaScript Algorithms and Data Structures Certification (freeCodeCamp, and 2023).',
 'projects': 'TaskMaster Web App-Built a full-stack task management app using React. js, Node. js, and MongoDB. Implemented user authentication and real-time notifications. Personal Portfolio Website-Designed and developed a personal website to showcase projects and blog posts. Focused on accessibility and responsive design.',
 'others': 'Other relevant participation includes Open-source contributor, and-Volunteer coding mentor at CodeClub Springfield.'}

job_description={'job info': 'Job Title Junior Web Developer. Location Remote (US-based preferred). Employment Type Full-Time.',
 'summary': 'We are seeking a passionate Junior Web Developer to join our growing team. This role is ideal for someone who enjoys building responsive, user-friendly web applications and working in a collaborative, agile environment. You will support front-end and back-end development tasks, contribute to code reviews, and help ensure our applications perform efficiently across devices.',
 'responsibilities': '- Develop and maintain responsive web pages using HTML5, CSS3, and JavaScript. - Work with React.js on the front end and Node.js  Express.js on the back end. - Design and consume RESTful APIs. - Collaborate with UIUX designers to implement mobile-first designs. - Use version control systems like Git and platforms such as GitHub. - Write clean, maintainable, and well-documented code. - Participate in Agile workflows including daily standups and sprint planning.',
 'requirements': '- Proficiency in HTML5, CSS3, and JavaScript (ES6). - Familiarity with React.js, Node.js, Express.js. - Basic understanding of MongoDB and MySQL. - Experience with Git, GitHub, VS Code, and Postman. - Comfortable working with APIs and debugging front-endback-end issues. - Strong communication and problem-solving skills. - Willingness to learn and grow in a fast-paced development environment.',
 'education': "Bachelor's degree in Computer Science or related field (or equivalent practical experience).",
 'preferred_qualifications': '- Internship or freelance experience building real-world web applications. - Exposure to tools like Figma, Firebase, or Vercel.'}

In [8]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [9]:
# llm_pipeline = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")

selected_resume, selected_jd = select_relevant_sections(resume, job_description)

summarized_resume = summarize_sections(selected_resume, summarizer)
summarized_jd = summarize_sections(selected_jd, summarizer)

# analyses = analyze_match(summarized_resume, summarized_jd, llm_pipeline)



Your max_length is set to 60, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)
Your max_length is set to 150, but your input_length is only 114. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Your max_length is set to 80, but your input_length is only 66. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)
Your max_length is set to 80, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


In [10]:
selected_resume

{'summary': 'Junior Web Developer with a strong foundation in front-end and back-end technologies. Skilled in building responsive, user-centric websites and applications. Passionate about learning new technologies and delivering high-quality code.',
 'skills': 'This candidate is proficient in Languages: HTML5, CSS3, JavaScript (ES6+), Python Frameworks/Libraries: React. js, Node. js, Express. js, Bootstrap Databases: MongoDB, MySQL Tools: Git, GitHub, Visual Studio Code, Figma, Postman Other: RESTful APIs, Responsive Design, and Agile Methodology.',
 'experience': 'Web Development Intern, BrightWave Solutions, Remote, Jan 2024-Apr 2024-Developed and maintained front-end components using React. js and Bootstrap. Built RESTful APIs with Node. js and Express. js. Created mobile-first responsive designs with UX teams. Improved website performance by 20%. Freelance Web Developer, Self-Employed, Remote, May 2023-Dec 2023-Designed and developed custom websites for local businesses. Integrated

In [11]:
summarized_resume

{'summary': ' Junior Web Developer with a strong foundation in front-end and back-end technologies . Skilled in building responsive, user-centric websites and applications . Passionate about learning new technologies and delivering high-quality code .',
 'skills': 'This candidate is proficient in Languages: HTML5, CSS3, JavaScript (ES6+), Python Frameworks/Libraries: React. js, Node. js, Express. js, Bootstrap Databases: MongoDB, MySQL Tools: Git, GitHub, Visual Studio Code, Figma, Postman Other: RESTful APIs, Responsive Design, and Agile Methodology.',
 'experience': ' Web Development Intern, BrightWave Solutions, Remote, Jan 2024-Apr 2024-Developed and maintained front-end components using React. js and Bootstrap . Created mobile-first responsive designs with UX teams . Improved website performance by 20% . Freelance Web Developer, Self-Employed, May 2023-Dec 2023, developed custom websites for local businesses .',
 'projects': ' TaskMaster Web App-Built a full-stack task management 

In [12]:
selected_jd

{'summary': 'We are seeking a passionate Junior Web Developer to join our growing team. This role is ideal for someone who enjoys building responsive, user-friendly web applications and working in a collaborative, agile environment. You will support front-end and back-end development tasks, contribute to code reviews, and help ensure our applications perform efficiently across devices.',
 'requirements': '- Proficiency in HTML5, CSS3, and JavaScript (ES6). - Familiarity with React.js, Node.js, Express.js. - Basic understanding of MongoDB and MySQL. - Experience with Git, GitHub, VS Code, and Postman. - Comfortable working with APIs and debugging front-endback-end issues. - Strong communication and problem-solving skills. - Willingness to learn and grow in a fast-paced development environment.',
 'responsibilities': '- Develop and maintain responsive web pages using HTML5, CSS3, and JavaScript. - Work with React.js on the front end and Node.js  Express.js on the back end. - Design and c

In [13]:
summarized_jd

{'summary': ' We are seeking a passionate Junior Web Developer to join our growing team . This role is ideal for someone who enjoys building responsive, user-friendly web applications .',
 'requirements': ' Proficiency in HTML5, CSS3, and JavaScript (ES6) Familiarity with React.js, Node.js and Express.js . Comfortable working with APIs and debugging front-end issues . Strong communication and problem-solving skills .',
 'responsibilities': ' - Develop and maintain responsive web pages using HTML5, CSS3, and JavaScript . - Use version control systems like Git and platforms such as GitHub . - Participate in Agile workflows including daily standups and sprint planning . - Collaborate with UIUX designers to implement mobile-first designs .',
 'preferred_qualifications': ' - Internship or freelance experience building real-world web applications . Exposure to tools like Figma, Firebase, or Vercel. - Exposure to Firebase .'}

In [14]:
def build_prompt(selected_resume, selected_jd):
    # Clean up the contact section to make it a simple sentence
    # contact_sentence = f"This resume belongs to {contact.replace('|', ',').strip().rstrip('.')}. Below are the key qualifications and experiences:\n"

    # Format resume sections
    resume_parts = [
        f"Summary: {selected_resume['summary']}\n",
        f"Skills: {selected_resume['skills']}\n",
        f"Experience: {selected_resume['experience']}\n",
        f"Projects: {selected_resume['projects']}\n"
    ]
    resume_text = "\n".join(resume_parts)

    # Format job description sections
    jd_parts = [
        f"Job Summary: {selected_jd['summary']}\n",
        f"Responsibilities: {selected_jd['responsibilities']}\n",
        f"Requirements: {selected_jd['requirements']}\n",
        f"Preferred Qualifications: {selected_jd['preferred_qualifications']}\n"
    ]
    jd_text = "\n".join(jd_parts)

    score=8

    # Final Prompt
    prompt = (
        # f"{contact_sentence}\n"
        f"--- Resume ---\n{resume_text}\n\n"
        f"--- Job Description ---\n{jd_text}\n\n"
        f"--- Semantic Score ---\nThe semantic score of this resume compare to the job description is {score}\n\n"
        # "Based on the resume and the job description above, assess the candidate's suitability for the role. "
        # "Highlight the most relevant qualifications, identify any skill gaps, and provide a short summary judgment."
        "--- Task ---\n"
        "Based on the resume, job description and their semantic score above, complete the following:\n"
        "1. In one sentence, explain why this score was given.\n"
        "3. In one sentence, suggest one specific improvement to strengthen the resume for this job.\n"
        # "Based on the resume and the job description above, give rating 1-10 about how strong this candidate's resume matching the job description."
    )

    return prompt

prompt = build_prompt(
    # contact=resume.get('contact', ''),
    selected_resume=summarized_resume,
    selected_jd=summarized_jd
)

print(prompt)


--- Resume ---
Summary:  Junior Web Developer with a strong foundation in front-end and back-end technologies . Skilled in building responsive, user-centric websites and applications . Passionate about learning new technologies and delivering high-quality code .

Skills: This candidate is proficient in Languages: HTML5, CSS3, JavaScript (ES6+), Python Frameworks/Libraries: React. js, Node. js, Express. js, Bootstrap Databases: MongoDB, MySQL Tools: Git, GitHub, Visual Studio Code, Figma, Postman Other: RESTful APIs, Responsive Design, and Agile Methodology.

Experience:  Web Development Intern, BrightWave Solutions, Remote, Jan 2024-Apr 2024-Developed and maintained front-end components using React. js and Bootstrap . Created mobile-first responsive designs with UX teams . Improved website performance by 20% . Freelance Web Developer, Self-Employed, May 2023-Dec 2023, developed custom websites for local businesses .

Projects:  TaskMaster Web App-Built a full-stack task management app 

In [15]:
# final_report = compile_analysis(resume['contact'], analyses)
# print(final_report)

#new

In [16]:
!pip install -q transformers accelerate


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


--- Resume ---
Summary:  Junior Web Developer with a strong foundation in front-end and back-end technologies . Skilled in building responsive, user-centric websites and applications . Passionate about learning new technologies and delivering high-quality code .

Skills: This candidate is proficient in Languages: HTML5, CSS3, JavaScript (ES6+), Python Frameworks/Libraries: React. js, Node. js, Express. js, Bootstrap Databases: MongoDB, MySQL Tools: Git, GitHub, Visual Studio Code, Figma, Postman Other: RESTful APIs, Responsive Design, and Agile Methodology.

Experience:  Web Development Intern, BrightWave Solutions, Remote, Jan 2024-Apr 2024-Developed and maintained front-end components using React. js and Bootstrap . Created mobile-first responsive designs with UX teams . Improved website performance by 20% . Freelance Web Developer, Self-Employed, May 2023-Dec 2023, developed custom websites for local businesses .

Projects:  TaskMaster Web App-Built a full-stack task management app 

In [18]:

output = generator(prompt, max_new_tokens=100, do_sample=False)
print(output[0]["generated_text"])


--- Resume ---
Summary:  Junior Web Developer with a strong foundation in front-end and back-end technologies . Skilled in building responsive, user-centric websites and applications . Passionate about learning new technologies and delivering high-quality code .

Skills: This candidate is proficient in Languages: HTML5, CSS3, JavaScript (ES6+), Python Frameworks/Libraries: React. js, Node. js, Express. js, Bootstrap Databases: MongoDB, MySQL Tools: Git, GitHub, Visual Studio Code, Figma, Postman Other: RESTful APIs, Responsive Design, and Agile Methodology.

Experience:  Web Development Intern, BrightWave Solutions, Remote, Jan 2024-Apr 2024-Developed and maintained front-end components using React. js and Bootstrap . Created mobile-first responsive designs with UX teams . Improved website performance by 20% . Freelance Web Developer, Self-Employed, May 2023-Dec 2023, developed custom websites for local businesses .

Projects:  TaskMaster Web App-Built a full-stack task management app 

In [19]:
output

[{'generated_text': "--- Resume ---\nSummary:  Junior Web Developer with a strong foundation in front-end and back-end technologies . Skilled in building responsive, user-centric websites and applications . Passionate about learning new technologies and delivering high-quality code .\n\nSkills: This candidate is proficient in Languages: HTML5, CSS3, JavaScript (ES6+), Python Frameworks/Libraries: React. js, Node. js, Express. js, Bootstrap Databases: MongoDB, MySQL Tools: Git, GitHub, Visual Studio Code, Figma, Postman Other: RESTful APIs, Responsive Design, and Agile Methodology.\n\nExperience:  Web Development Intern, BrightWave Solutions, Remote, Jan 2024-Apr 2024-Developed and maintained front-end components using React. js and Bootstrap . Created mobile-first responsive designs with UX teams . Improved website performance by 20% . Freelance Web Developer, Self-Employed, May 2023-Dec 2023, developed custom websites for local businesses .\n\nProjects:  TaskMaster Web App-Built a ful

#Were fine tuning babe

In [22]:
print("""prompt": "Resume: ... \nJob Description: ... \nSemantic Score: 8\nTask: Rate and comment", "completion": "1. This candidate scores 8 because... 2. Improvement suggestion: ... 3. Alignment: ...""")

prompt": "Resume: ... 
Job Description: ... 
Semantic Score: 8
Task: Rate and comment", "completion": "1. This candidate scores 8 because... 2. Improvement suggestion: ... 3. Alignment: ...


**GENERATE TRAINING DATA**